In [9]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import java.io._
import scala.util.Try
import java.net.URLDecoder.decode

val spark = SparkSession
    .builder
    .master("yarn")
    .appName("laba02_autousers_VasevVS")
    .getOrCreate

val path_users = "/labs/laba02/autousers.json"
val path_domains = "/labs/laba02/logs"

val userDf = spark.read.json(path_users)
val users = userDf
      .select(
        explode(col("autousers"))
        .as("UID"), lit(1).alias("Auto_flag"))

val decoding = udf { (url: String) =>
    Try(decode(url, "UTF-8")).toOption.getOrElse("No_host")
  }

val domains = spark.read.options(Map("delimiter"->"\t")).csv(path_domains)
    .select(col("_c0").alias("UID"), col("_c2").alias("URL"))
    .filter("URL like 'http%'")
    .withColumn("URL", decoding(col("URL")))
    .withColumn("URL", regexp_replace(col("URL"), "(https://www.|http://www.|https://|http://|[/].*)", ""))
    .join(broadcast(users), Seq("UID"), "left")
    .cache()

val count_1 = domains.filter(col("Auto_flag").isNotNull).count()

val result = domains
    .groupBy("URL")
    .agg(((pow(sum("Auto_flag"),2)/count("URL")).cast(DecimalType(38,20))/count_1).cast(DecimalType(38,20)).alias("otiai"))
    .sort(col("otiai").desc, col("URL").asc)
    .take(200)
val pw = new PrintWriter(new FileOutputStream(new File("/data/home/vladimir.vasev/laba02_domains.txt" ),true))
  
for (ans <- result){
    pw.append(ans.mkString("\t") + "\n")
  }
pw.close();


spark = org.apache.spark.sql.SparkSession@11915afc
path_users = /labs/laba02/autousers.json
path_domains = /labs/laba02/logs
userDf = [autousers: array<string>]
users = [UID: string, Auto_flag: int]
decoding = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))
domains = [UID: string, URL: string ... 1 more field]
count_1 = 313527


result: Array[org.apache.spa...


313527